In [2]:
import requests
import json
from cis_requests import *
from config import *

In [4]:
c = config_from_file("../dev_config.json")

In [8]:
sample = "Hi this is an email from Mike to Jim talking about how we should get those records down to region 5 and then send them to region 10. Thanks!"

In [9]:
deep_detect_classify(sample, c)

{'prob': 0.8488791584968567, 'last': True, 'cat': '401-1006-b'}

In [10]:
nlp_buddy_analyze(sample, c)['summary']

' Hi this is an email from Mike to Jim talking about how we should get those records down to region 5 and then send them to region 10. Thanks!'

In [39]:
sample_doc = open('test.docx', 'rb')

In [40]:
doc_text = tika(sample_doc, c)

In [41]:
sample_doc.close()

In [42]:
sample_pdf = open("test.pdf", 'rb')

In [43]:
pdf_text = tika(sample_pdf, c)

In [44]:
sample_pdf.close()

In [18]:
sample_doc = open('test.docx', 'rb')
files = {'file': sample_doc}
server = 'http://localhost:8000/record_schedule_prediction'
r = requests.post(server, files=files)
r.json()

{'body': {'predictions': [{'classes': [{'cat': '401-1006-b',
      'last': True,
      'prob': 0.938021719455719}],
    'uri': '0'}]},
 'head': {'method': '/predict', 'service': 'records', 'time': 37.0},
 'status': {'code': 200, 'msg': 'OK'}}

In [19]:
sample_pdf = open("test.pdf", 'rb')
files = {'file': sample_pdf}
server = 'http://localhost:8000/record_schedule_prediction'
r = requests.post(server, files=files)
r.json()

{'body': {'predictions': [{'classes': [], 'uri': '0'}]},
 'head': {'method': '/predict', 'service': 'records', 'time': 36.0},
 'status': {'code': 200, 'msg': 'OK'}}

In [51]:
deep_detect_classify(doc_text, c)

{'status': {'code': 200, 'msg': 'OK'},
 'head': {'method': '/predict', 'service': 'records', 'time': 36.0},
 'body': {'predictions': [{'classes': [{'prob': 0.938021719455719,
      'last': True,
      'cat': '401-1006-b'}],
    'uri': '0'}]}}

In [50]:
deep_detect_classify(pdf_text, c, threshold=0.01)

{'status': {'code': 200, 'msg': 'OK'},
 'head': {'method': '/predict', 'service': 'records', 'time': 37.0},
 'body': {'predictions': [{'classes': [{'prob': 0.10436568409204483,
      'last': True,
      'cat': '108-1035-c'}],
    'uri': '0'}]}}

In [47]:
doc_text

'[bookmark: _GoBack]This is a test Word file to show how Tika can extract text.\n'

In [48]:
pdf_text

'\nThis is a test Word file to show how Tika can extract text. \n\n\n'